



cc







In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import seaborn as sns
from ipywidgets import interact 
from matplotlib import pyplot as plt

from datetime import datetime
from scipy import stats as st
import json
from pandas import json_normalize
from ipywidgets import interact


In [2]:
from download import download

ModuleNotFoundError: No module named 'download'

In [17]:
#Database import 
url = [
#Celleneuve
    'https://data.montpellier3m.fr/sites/default/files/ressources/MMM_EcoCompt_X2H20042633_archive.json',
#Lattes2
    'https://data.montpellier3m.fr/sites/default/files/ressources/MMM_EcoCompt_X2H20042634_archive.json',

path_target0 = "./data_visulization/Celleneuve.json"
path_target1 = "./data_visulization/Lattes2.json"
download(url[0], path_target0, replace=False)
download(url[1], path_target1, replace=False)

SyntaxError: invalid syntax (<ipython-input-17-29f95f9f9b2d>, line 9)